In [1]:
import pyspark

import os
from datetime import date
import functools
from IPython.core.display import display, HTML
#import findspark
#findspark.init()
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T

from pyspark.sql.functions import to_timestamp, count, isnan, mean, col, countDistinct, format_number, dayofmonth, hour, dayofyear, month, year, weekofyear, date_format, when,datediff, months_between

import re 
from functools import reduce

In [3]:
spark = SparkSession.builder.appName("SRAG").getOrCreate()

# Read data

In [34]:
df = spark.read.csv('/Volumes/My Passport/PAMEpi_datalake/raw_data_covid19_version-21-12-07/data-sindrome_respiratoria_aguda_grave_incluindo_covid/INFLUD-29-11-2021.csv',sep=';', header = True, encoding="utf-8", inferSchema = True)

In [35]:
df1 = spark.read.csv('/Volumes/My Passport/PAMEpi_datalake/raw_data_covid19_version-21-12-07/data-sindrome_respiratoria_aguda_grave_incluindo_covid/INFLUD21-29-11-2021.csv',sep=';', header = True, encoding="utf-8", inferSchema = True)

In [7]:
df.count()

1199699

In [11]:
df1.count()

1625471

In [12]:
df.count() + df1.count()

2825170

In [8]:
df.limit(5).toPandas() 

,DT_NOTIFIC,SEM_NOT,DT_SIN_PRI,SEM_PRI,SG_UF_NOT,ID_REGIONA,CO_REGIONA,ID_MUNICIP,CO_MUN_NOT,ID_UNIDADE,...,DS_AN_OUT,TP_AM_SOR,SOR_OUT,DT_CO_SOR,TP_SOR,OUT_SOR,DT_RES,RES_IGG,RES_IGM,RES_IGA
0,07/01/2020,2,03/01/2020,1,DF,None,NaN,BRASILIA,530010,HRT HOSPITAL REGIONAL DE TAGUATINGA,...,None,NaN,None,None,NaN,None,None,NaN,NaN,NaN
1,10/01/2020,2,07/01/2020,2,DF,None,NaN,BRASILIA,530010,HOSPITAL MATERNO INFANTIL DR ANTONIO LISBOA,...,None,NaN,None,None,NaN,None,None,NaN,NaN,NaN
2,13/02/2020,7,08/02/2020,6,SP,GVE XXXI SOROCABA,1353.0,SALTO,354520,HOSPITAL E MATERNIDADE MUNICIPAL N S DO MONTE ...,...,None,NaN,None,None,NaN,None,None,NaN,NaN,NaN
3,27/01/2020,5,26/01/2020,5,CE,1 CRES FORTALEZA,1519.0,FORTALEZA,230440,HIAS HOSPITAL INFANTIL ALBERT SABIN,...,None,NaN,None,None,NaN,None,None,NaN,NaN,NaN
4,16/01/2020,3,10/01/2020,2,SP,GVE XXXIII TAUBATE,1352.0,TAUBATE,355410,HOSPITAL REGIONAL DO VALE DO PARAIBA,...,None,NaN,None,None,NaN,None,None,NaN,NaN,NaN


In [9]:
df.printSchema()

root
 |-- DT_NOTIFIC: string (nullable = true)
 |-- SEM_NOT: integer (nullable = true)
 |-- DT_SIN_PRI: string (nullable = true)
 |-- SEM_PRI: integer (nullable = true)
 |-- SG_UF_NOT: string (nullable = true)
 |-- ID_REGIONA: string (nullable = true)
 |-- CO_REGIONA: integer (nullable = true)
 |-- ID_MUNICIP: string (nullable = true)
 |-- CO_MUN_NOT: integer (nullable = true)
 |-- ID_UNIDADE: string (nullable = true)
 |-- CO_UNI_NOT: integer (nullable = true)
 |-- CS_SEXO: string (nullable = true)
 |-- DT_NASC: string (nullable = true)
 |-- NU_IDADE_N: integer (nullable = true)
 |-- TP_IDADE: integer (nullable = true)
 |-- COD_IDADE: string (nullable = true)
 |-- CS_GESTANT: integer (nullable = true)
 |-- CS_RACA: integer (nullable = true)
 |-- CS_ETINIA: string (nullable = true)
 |-- CS_ESCOL_N: integer (nullable = true)
 |-- ID_PAIS: string (nullable = true)
 |-- CO_PAIS: integer (nullable = true)
 |-- SG_UF: string (nullable = true)
 |-- ID_RG_RESI: string (nullable = true)
 |-- CO

In [32]:
df_concat= df.unionByName(df1, allowMissingColumns=True)

In [33]:
df_concat.limit(5).toPandas()

,DT_NOTIFIC,SEM_NOT,DT_SIN_PRI,SEM_PRI,SG_UF_NOT,ID_REGIONA,CO_REGIONA,ID_MUNICIP,CO_MUN_NOT,ID_UNIDADE,...,RES_IGM,RES_IGA,ESTRANG,VACINA_COV,DOSE_1_COV,DOSE_2_COV,LAB_PR_COV,LOTE_1_COV,LOTE_2_COV,FNT_IN_COV
0,None,2,None,1,DF,None,NaN,BRASILIA,530010,HRT HOSPITAL REGIONAL DE TAGUATINGA,...,NaN,NaN,NaN,None,None,None,None,None,None,NaN
1,None,2,None,2,DF,None,NaN,BRASILIA,530010,HOSPITAL MATERNO INFANTIL DR ANTONIO LISBOA,...,NaN,NaN,NaN,None,None,None,None,None,None,NaN
2,None,7,None,6,SP,GVE XXXI SOROCABA,1353.0,SALTO,354520,HOSPITAL E MATERNIDADE MUNICIPAL N S DO MONTE ...,...,NaN,NaN,NaN,None,None,None,None,None,None,NaN
3,None,5,None,5,CE,1 CRES FORTALEZA,1519.0,FORTALEZA,230440,HIAS HOSPITAL INFANTIL ALBERT SABIN,...,NaN,NaN,NaN,None,None,None,None,None,None,NaN
4,None,3,None,2,SP,GVE XXXIII TAUBATE,1352.0,TAUBATE,355410,HOSPITAL REGIONAL DO VALE DO PARAIBA,...,NaN,NaN,NaN,None,None,None,None,None,None,NaN


# Convert date colunms to date

In [30]:
df_concat.printSchema()

root
 |-- DT_NOTIFIC: date (nullable = true)
 |-- SEM_NOT: integer (nullable = true)
 |-- DT_SIN_PRI: date (nullable = true)
 |-- SEM_PRI: integer (nullable = true)
 |-- SG_UF_NOT: string (nullable = true)
 |-- ID_REGIONA: string (nullable = true)
 |-- CO_REGIONA: integer (nullable = true)
 |-- ID_MUNICIP: string (nullable = true)
 |-- CO_MUN_NOT: integer (nullable = true)
 |-- ID_UNIDADE: string (nullable = true)
 |-- CO_UNI_NOT: integer (nullable = true)
 |-- CS_SEXO: string (nullable = true)
 |-- DT_NASC: date (nullable = true)
 |-- NU_IDADE_N: integer (nullable = true)
 |-- TP_IDADE: integer (nullable = true)
 |-- COD_IDADE: string (nullable = true)
 |-- CS_GESTANT: integer (nullable = true)
 |-- CS_RACA: integer (nullable = true)
 |-- CS_ETINIA: string (nullable = true)
 |-- CS_ESCOL_N: integer (nullable = true)
 |-- ID_PAIS: string (nullable = true)
 |-- CO_PAIS: integer (nullable = true)
 |-- SG_UF: string (nullable = true)
 |-- ID_RG_RESI: string (nullable = true)
 |-- CO_RG_RE

In [27]:
list_var_date = ['DT_NOTIFIC', 'DT_SIN_PRI', 'DT_NASC', 'DT_INTERNA','DT_ENTUTI','DT_SAIDUTI','DT_COLETA','DT_ENCERRA']

In [29]:
for col in list_var_date:
    df_concat = df_concat.withColumn(col, F.col(col).cast(T.DateType()))